# Softmax Regression

One-Hot encoding: class 중에서 하나만 hot하게 해준다고 표현한다. 하나만 1이고 나머지는 0으로 encoding

In [4]:
import tensorflow as tf

In [11]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])

#Class의 갯수만큼 weight와 b의 갯수가 달라진다.
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

y_ = tf.matmul(X, W) + b
#softmax function
#hypothesis = tf.exp(y_)/tf.reduce_sum(tf.exp(y_))
hypothesis = tf.nn.softmax(y_)

#cost function : cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis = 1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict = {X: x_data, Y: y_data}))
            
#testing & one-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))
    print('-'*50)
    #argmax함수는 벡터를 넣으면 값이 가장 높은 좌표를 알려준다.
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))
    print('-'*50)
    
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))
    print('-'*50)
    
    all = sess.run(hypothesis, feed_dict = {X:[[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))
    

0 4.23834
200 0.492255
400 0.402618
600 0.324751
800 0.255954
1000 0.229082
1200 0.207937
1400 0.190308
1600 0.17537
1800 0.162548
2000 0.151423
[[  6.42104959e-03   9.93572712e-01   6.31955299e-06]] [1]
--------------------------------------------------
[[ 0.83953452  0.15178913  0.00867637]] [0]
--------------------------------------------------
[[  6.79004764e-09   2.78427295e-04   9.99721587e-01]] [2]
--------------------------------------------------
[[  6.42104959e-03   9.93572712e-01   6.31955299e-06]
 [  8.39534521e-01   1.51789129e-01   8.67637247e-03]
 [  6.79004764e-09   2.78427266e-04   9.99721587e-01]] [1 0 2]


# Fancy Softmax

In [1]:
import numpy as np
xy = np.loadtxt('/Users/kjw1o/Documents/dev/DeepLearningZeroToAll/data-04-zoo.csv', delimiter = ',', dtype = np.float32)

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)
nb_classes = 7 # 0~6의 값을 가짐

X = tf.placeholder(tf.float32, shape=[None, 16])
Y = tf.placeholder(tf.int32, shape=[None, 1])
#Y의 one_hot encoding 실행 클래스의 갯수가 필요하다.
Y_one_hot = tf.one_hot(Y, nb_classes)
#one hot만하면 shape가 (?, 1, 7)가 되어서 3차원이 되버리므로 아래와같이 reshape으로 2차원으로 바꿔준다.
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

#Class의 갯수만큼 weight와 b의 갯수가 달라진다.
W = tf.Variable(tf.random_normal([16, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')


#logits는 y-bar네요
logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

#cross entropy cost/Loss 위와같이 계산하는 방법도 있지만 tf에 이미 있다.
# ybar인 logits를 argument로 필요
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

#정확도 분석 
#예측값 
predicted = tf.argmax(hypothesis, 1)
#tf.argmax(Y_one_hoy, 1)인 실제값과 predicted 예측값을 비교한다.
correct_prediction = tf.equal(predicted, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict = {X: x_data, Y: y_data})
            print("Step : {:5}\t Loss: {:.3f}\t Acc : {:.2%}".format(step, loss, acc))
            
    #실제 적용
    pred = sess.run(predicted, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] prediction: {} True Y : {}".format(p == int(y), p, int(y)))
    y_shape, y_one_shape = sess.run([Y, Y_one_hot], feed_dict = {X: x_data, Y: y_data })
    print("Y : {} \n Y_one_hot: {}".format(y_shape, y_one_shape))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kjw1o/Documents/dev/DeepLearningZeroToAll/data-04-zoo.csv'